In [1]:
from typing import Tuple

import pandas

from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'uk'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
# param_grid = {  
#     'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
#     'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
# }

model = Prophet().fit(df_train)

03:01:45 - cmdstanpy - INFO - Chain [1] start processing
03:01:45 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [9]:
df_predicted, rmse = test_model(df_test, model)

In [10]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-01-01,3.575879,2.355865,4.865763,3.575879,3.575879,0.019078,0.019078,0.019078,0.019078,0.019078,0.019078,0.0,0.0,0.0,3.594957
1,2022-02-01,3.546053,2.344525,4.752420,3.546053,3.546053,0.007937,0.007937,0.007937,0.007937,0.007937,0.007937,0.0,0.0,0.0,3.553990
2,2022-03-01,3.519113,2.273448,4.805330,3.519113,3.519113,0.010264,0.010264,0.010264,0.010264,0.010264,0.010264,0.0,0.0,0.0,3.529377
3,2022-04-01,3.489286,2.198471,4.752952,3.489189,3.489915,0.019840,0.019840,0.019840,0.019840,0.019840,0.019840,0.0,0.0,0.0,3.509126
4,2022-05-01,3.460421,2.232876,4.750090,3.459432,3.462883,0.007446,0.007446,0.007446,0.007446,0.007446,0.007446,0.0,0.0,0.0,3.467868
5,2022-06-01,3.430595,2.215451,4.691335,3.427590,3.435244,0.009444,0.009444,0.009444,0.009444,0.009444,0.009444,0.0,0.0,0.0,3.440038
6,2022-07-01,3.401730,2.265896,4.623765,3.396033,3.410527,0.017672,0.017672,0.017672,0.017672,0.017672,0.017672,0.0,0.0,0.0,3.419402
7,2022-08-01,3.371903,2.186667,4.670239,3.363023,3.384657,0.021095,0.021095,0.021095,0.021095,0.021095,0.021095,0.0,0.0,0.0,3.392998
8,2022-09-01,3.342077,2.190499,4.587433,3.330043,3.358595,0.040037,0.040037,0.040037,0.040037,0.040037,0.040037,0.0,0.0,0.0,3.382114
9,2022-10-01,3.313212,2.062752,4.699230,3.297110,3.336100,0.020319,0.020319,0.020319,0.020319,0.020319,0.020319,0.0,0.0,0.0,3.333531


In [11]:
rmse

0.2853295127529523

In [12]:
with open(f'{country}_prophet_base_model.json', 'w') as f:
    f.write(model_to_json(model))

In [13]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [14]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [15]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,3.167927,1.992653,4.440137,3.167927,3.167927,0.034669,0.034669,0.034669,0.034669,0.034669,0.034669,0.0,0.0,0.0,3.202596
1,2023-04-01,3.138101,1.873933,4.451514,3.138101,3.138101,0.025588,0.025588,0.025588,0.025588,0.025588,0.025588,0.0,0.0,0.0,3.163689
2,2023-05-01,3.109236,1.884592,4.339369,3.109236,3.109236,-0.001015,-0.001015,-0.001015,-0.001015,-0.001015,-0.001015,0.0,0.0,0.0,3.108221
